# 顔文字生成器（サンプル）
単純ニューラルネットワークだけのシンプル構成

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

## 前処理

### データ読み込み

In [19]:
# 特殊文字
sp = {'pad': '<PAD>',
      # 'bos': '<BOS>',
      # 'eos': '<EOS>',
      'unk': '<UNK>'}
# pad : padding. 文字列長を一定にするために使う
# bos : begin of sequence. 文頭文字．Decoderの最初の入力
# eos : end of sequence. 文末文字．
# unk : unknown. 出現数が低いものに割り当てる

In [20]:
sp.values()

dict_values(['<PAD>', '<UNK>'])

In [21]:
KAOMOJI_MAX = 10    # 顔文字最大長

kmj_list = []   # 顔文字リスト
len_list = []       # <BOS> から <EOS> までの文字数のリスト
char_list = []      # 顔文字に使用されている文字のリスト

char_list += list(sp.values())
file_name = 'kaomoji_MAX=' + str(KAOMOJI_MAX) + '_DA.txt'

with open(file_name, mode='r') as file:
  for line in file:
    # temp = [sp['bos']]
    temp = list(line.replace('\n', ''))
    # temp += [sp['eos']]
    len_list.append(len(temp))
    temp += [sp['pad'] for _ in range(KAOMOJI_MAX - len(temp))]
    kmj_list.append(temp)
    char_list += temp

# 重複を消す
char_list = sorted(set(char_list), key=char_list.index)

In [22]:
print(kmj_list[0])

['(', '’', '⌒', '’', ')', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [23]:
print('Number of kaomoji  :', len(kmj_list))
print('Number of character:', len(char_list))

Number of kaomoji  : 10300
Number of character: 719


### 出現数が少ないものを置換

In [24]:
# 最小出現数
MIN_APPEAR = 20

kmj_list = np.array(kmj_list)

cnt = 0
for c in char_list:
  mask = (kmj_list == c)
  if np.sum(mask) < MIN_APPEAR:
    kmj_list[mask] = sp['unk']

char_list = list(sp.values()) + kmj_list.flatten().tolist()
char_list = sorted(set(char_list), key=char_list.index)

In [25]:
print('Number of character:', len(char_list))

Number of character: 200


### 添字検索
顔文字に使われる文字が文字リストの何番目にあるか調べる

In [26]:
kmj_index = []    # 添字リスト

for kmj in kmj_list.tolist():
  temp = [char_list.index(c) for c in kmj]
  kmj_index.append(temp)

In [27]:
print(len_list[0])
kmj_index[0]

5


[2, 3, 4, 3, 5, 0, 0, 0, 0, 0]

### One-hotベクトル化



In [28]:
kmj_num = len(kmj_index)        # 顔文字の総数
kmj_size = len(kmj_index[0])    # 1つの顔文字の長さ
char_num = len(char_list)       # 文字の種類数

# One-hotベクトルリスト
kmj_onehot = np.zeros((kmj_num, kmj_size, char_num))

for i, index in enumerate(kmj_index):
  mask = range(char_num) == np.array(index).reshape((kmj_size, 1))
  kmj_onehot[i][mask] = 1

In [29]:
kmj_onehot.shape

(10300, 10, 200)

### 訓練・検証・テスト用に分ける

In [30]:
dataset = torch.utils.data.TensorDataset(
  torch.tensor(kmj_onehot.astype('float32')),
  torch.tensor(len_list)
)

In [31]:
train_size = int(len(dataset) * 0.85)
valid_size = int(len(dataset) * 0.10)
test_size  = len(dataset) - train_size - valid_size

# indices = np.arange(len(dataset))

# dataset_train = torch.utils.data.Subset(dataset, indices[:train_size])
# dataset_valid = torch.utils.data.Subset(dataset, indices[train_size:train_size+valid_size])
# dataset_test  = torch.utils.data.Subset(dataset, indices[train_size+valid_size:])

split = [train_size, valid_size, test_size]

dataset_train, dataset_valid, dataset_test = torch.utils.data.random_split(dataset, split)

In [32]:
train_size

8755

In [33]:
batch_size = 128

dataloader_train = torch.utils.data.DataLoader(
  dataset_train,
  batch_size=batch_size,
  shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
  dataset_valid,
  batch_size=batch_size,
  shuffle=True
)

In [34]:
for x, len_seq in dataloader_train:
  print(x.shape, len_seq.shape)
  break

torch.Size([128, 10, 200]) torch.Size([128])


## 学習

### ネットワークの定義

In [35]:
class SimpleNet(nn.Module):
  def __init__(self, in_dim, hid_dim):
    super().__init__()
    self.encoder = nn.Linear(in_dim, hid_dim)
    self.decoder = nn.Linear(hid_dim, in_dim)

  def forward(self, x):
    x = self.encoder(x)
    return self.decoder(x)

### 学習

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [81]:
n_epochs = 10

N = KAOMOJI_MAX
char_num = len(char_list)
hid_dim = 32

net = SimpleNet(char_num, hid_dim)

optimizer = optim.Adam(net.parameters())    # 最適化
criterion = nn.CrossEntropyLoss()           # 損失関数

In [82]:
for epoch in range(n_epochs):
  losses_train = []
  losses_valid = []
  acc_train = 0
  acc_valid = 0

  # 訓練
  net.train()
  for x, len_seq in dataloader_train:
    net.zero_grad()  # 勾配の初期化

    y = net(x)

    loss = criterion(y, x)
    acc_train += (y.argmax(dim=2) == x.argmax(dim=2)).sum()
    loss.backward()  # 誤差の逆伝播
    losses_train.append(loss.tolist())

    optimizer.step()  # パラメータの更新

  # 検証
  net.eval()
  for x, len_seq in dataloader_valid:
    y = net(x)

    loss = criterion(y, x)
    acc_valid += (y.argmax(dim=2) == x.argmax(dim=2)).sum()
    losses_valid.append(loss.tolist())

  if (epoch+1) % 1 == 0:
    print('EPOCH: {:>3}, Train Loss: {:>4.5f}  Acc: {:>.3f},    Valid Loss: {:>4.5f}  Acc: {:>.3f}'.format(
        epoch+1,
        np.mean(losses_train),
        acc_train / (train_size * KAOMOJI_MAX),
        np.mean(losses_valid),
        acc_valid / (valid_size * KAOMOJI_MAX)
    ))

EPOCH:   1, Train Loss: 0.10841  Acc: 0.384,    Valid Loss: 0.10035  Acc: 0.676
EPOCH:   2, Train Loss: 0.08771  Acc: 0.890,    Valid Loss: 0.07318  Acc: 0.977
EPOCH:   3, Train Loss: 0.05912  Acc: 0.984,    Valid Loss: 0.04714  Acc: 0.989
EPOCH:   4, Train Loss: 0.03941  Acc: 0.991,    Valid Loss: 0.03385  Acc: 0.992
EPOCH:   5, Train Loss: 0.03084  Acc: 0.994,    Valid Loss: 0.02819  Acc: 0.994
EPOCH:   6, Train Loss: 0.02716  Acc: 0.997,    Valid Loss: 0.02583  Acc: 0.998
EPOCH:   7, Train Loss: 0.02529  Acc: 0.999,    Valid Loss: 0.02446  Acc: 0.998
EPOCH:   8, Train Loss: 0.02425  Acc: 0.999,    Valid Loss: 0.02360  Acc: 1.000
EPOCH:   9, Train Loss: 0.02364  Acc: 1.000,    Valid Loss: 0.02303  Acc: 1.000
EPOCH:  10, Train Loss: 0.02326  Acc: 1.000,    Valid Loss: 0.02308  Acc: 1.000


## 評価

In [83]:
def convert_str(x):
  x = np.array(char_list)[x.argmax(dim=1)]
  x = [c for c in x if c not in sp.values()]

  return ''.join(x)

def generate(net, base=None, rate=0.0, mean=0.0, std=1.0):
  if base is None:
    z = torch.normal(mean=mean, std=std, size=(N, hid_dim))
  else:
    z = net.encoder(base.unsqueeze(0))
    eps = 2 * rate * torch.rand(1, hid_dim) - rate
    z = z + eps

  y = net.decoder(z)
  gen = convert_str(y.squeeze(0))

  return ''.join(gen)

In [84]:
for i in np.random.randint(0, len(dataset_test), size=10):
  test = dataset_test[i][0]
  print('base     :', convert_str(test))
  print('generate :', generate(net, base=test, rate=0.0))

base     : (’-’)
generate : (’-’)
base     : (。・・)σ
generate : (。・・)σ
base     : ★⌒(　v'b◆)
generate : ★⌒(　v'b◆)
base     : ー(・∀・)
generate : ー(・∀・)
base     : ☆(>x<)
generate : ☆(>x<)
base     : ┃∀`*)ノ゙♪
generate : ┃∀`*)ノ゙♪
base     : _(・・)/◆
generate : _(・・)/◆
base     : (　зз)ノ
generate : (　зз)ノ
base     : (　^ิ艸^ิ゚)
generate : (　^ิ艸^ิ゚)
base     : ○_彡(^o^　)
generate : ○_彡(^o^　)


In [85]:
for _ in range(10):
  print('generate :', generate(net, base=None))

generate : ゝ▼oд、メΣ┓óд
generate : シ|-;`v◇⌒(∀
generate : =(ッシっΘノ┐σσ
generate : ×∇∇〇“ゝ/□εc
generate : ヾ)◇ゝつノヽΘ﻿
generate : ∀Д゚▼;|Σ゙ó~
generate : ヾv】_<(φ\,~
generate : O]♪+́ω┛∞_
generate : 皿ヾ¬△*┓　∇゚ゝ
generate : Σ_\*θΘ≡〃ゞ


In [86]:
def similar(c, net, num=5):
  index = char_list.index(c)
  weight = net.encoder(torch.tensor(np.eye(char_num, dtype='float32')))
  vector = weight[index]
  diff = weight - vector
  norm = torch.norm(diff, dim=1).detach().numpy()
  for _ in range(num):
    min_index = norm.argmin()
    print('{} : {}'.format(char_list[min_index], norm[min_index]))
    norm[min_index] = 100

In [87]:
similar('(', net)

( : 0.0
✿ : 2.416447639465332
☝ : 2.4338538646698
∫ : 2.4672160148620605
′ : 2.4785420894622803


## 感想

このネットワークは，  
Encoder：1層のニューラルネットワーク（$ Wx+b $ のやつ）  
Decoder：1層のニューラルネットワーク（$ Wx+b $ のやつ）  
のシンプルな構成

入力：(＾ー＾) としたらとき  
出力：(＾ー＾) となるからできているように見えるが，  
実体は，1文字に対して1文字が出ているだけ．  
要は，顔文字を分解した1文字ずつが，  
入力："（"　　------->　　出力："（"  
入力："＾"　　------->　　出力："＾"  
入力："ー"　　------->　　出力："ー"  
入力："＾"　　------->　　出力："＾"  
入力："）"　　------->　　出力："）"  
となっているだけで，顔文字でなくとも再現可能である．

1文字に対して固有の潜在ベクトルがある．
文字をOne-hotベクトルに変換してから入力しているため，入力の次元は大きくなるが，オートエンコーダを通して次元圧縮はできている．

また，1文字の潜在ベクトル同士の距離を計算すると，例えば「(」であれば一番近いものは「)」となり，なんとなくできている感じもある．

